In [ ]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
!pip install pmdarima
import statsmodels.api as sm
from pmdarima.arima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [2]:
sales_data = pd.read_excel("../input/sales-data/sales_data.xlsx")
sales_data.head()

,transaction id,product id,product description,quantity sold,transaction timestamp,unit price,customer id,transaction country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
date = pd.read_excel("../input/sales-data/date.xlsx")
date.head()

In [ ]:
sales_data.shape, date.shape

In [ ]:
print(len(sales_data["transaction country"].unique()))
sales_data["transaction country"].unique()

In [ ]:
len(sales_data["product description"].unique())

In [ ]:
df = [sales_data, date]
sales = pd.concat(df, axis=1, sort=False)
sales.head()

**Remove Duplicates:**

In [ ]:
print('sales_data duplicated:{}'.format(sales.duplicated().sum()))
sales.drop_duplicates(inplace=True)

In [ ]:
print('sales_data duplicated:{}'.format(sales.duplicated().sum()))

**Remove the negative quantity sold from sales dataframe:**

In [ ]:
sales[sales["quantity sold"]< 0].shape

In [ ]:
sales.drop(sales[sales["quantity sold"]< 0].index, inplace=True)

In [ ]:
sales[sales["quantity sold"]< 0].shape

**First order and last order date**

In [ ]:
print("Starting Date : ", min(sales.date))
print("Ending Date : ", max(sales.date))

In [ ]:
# monthly_sales=df.groupby(["transaction country", "transaction id","customer id"])[
#    "date","unit price","quantity sold"].agg({"date":["min",'max'],"unit price":"mean","quantity sold":"sum"})
    
# monthly_sales.head()

In [ ]:
sales.columns

**Remove timestamp columns because we are not going to use.**

In [ ]:
sales.drop(['timestamp              ', "transaction timestamp", "time_of_day(hh:mm:ss)"], axis=1, inplace=True)

In [ ]:
sales.sample(3)

In [ ]:
sales.columns

# Feature creation
**Here we are multiplying the quatity sold and unit price.**

In [ ]:
sales["total"] = sales["quantity sold"] * sales["unit price"]
sales.head()

**Groupby the sales by date with total and the total should be added because it shows the total money made by company in that sale.**

Here I used the resampling because it shows clearly that the total is incresing or decresing.

In [ ]:
datewise_total = sales.groupby("date").agg({"total":"sum"})
datewise_total.head(2)

In [ ]:
datewise_total.boxplot()

In [ ]:
datewise_total = datewise_total.resample("M").sum()
datewise_total.head()

Graph : 

[total = quantity sold * unit price]

date vs total    

**Groupby the sales dataframe by date. Quantity sold should be added because it shows the total product sold in that day. **

In [ ]:
datewise_total.plot(figsize=(10, 5))

In [ ]:
# datewise_total = sales.groupby(["date"])["date", "total"].agg("sum")
datewise_total = sales.groupby(["date"])["date", "total"].agg("sum")
datewise_total = datewise_total.resample("M").sum()
#datewise_total["total"].fillna(method="ffill",inplace=True)
datewise_total.head()

In [ ]:
plt.matshow(datewise_total)
plt.show()

In [ ]:
seas_d=sm.tsa.seasonal_decompose(datewise_total['total'],model="additive",period=2)
fig=seas_d.plot()
fig.set_figheight(15)
fig.set_figwidth(10)
plt.show()

In [ ]:
datewise_quantity = sales.groupby(["date"]).agg({"quantity sold":"sum"})
datewise_quantity = datewise_quantity.resample("M").sum()
datewise_quantity.head()

In [ ]:
datewise_quantity.tail()

Graph :
 
date vs quantity sold

In [ ]:
datewise_quantity.plot(figsize=(15,5))

You can conclude from above graph. The max sales is increasing day by day, but it variable. The sales are cleary depend upon the other factors like festivals or something. It dramatically decreasing in the last month i.e. december.

In [ ]:
day_total = sales.groupby(["day_name"]).agg({"total":"sum"})
day_total.sort_values("total")

In [ ]:
day_total.plot.bar(figsize=(15,5))

From above graph it is clear that the sales are also depend upon the day. If you see at Sunday the sales are cleary low compares to other days.

In [ ]:
country_total = sales.groupby("transaction country")["transaction country", "total"].agg("sum")
country_total.head()

In [ ]:
country_total.plot.bar(figsize=(25, 5))

In [ ]:
sales.columns

In [ ]:
customer_country = sales.groupby(["transaction country"])["transaction country","customer id"].agg("nunique")
customer_country.drop(["transaction country"], inplace=True, axis=1)
customer_country.head()

In [ ]:
customer_country.plot.bar(figsize=(25, 5))

From above graph it is clear that the much greater number of customers belong to United Kingdom compares
to other countries. 



**Countrywise growth of sales:**



In [ ]:
uk_sales = sales[sales["transaction country"]=="United Kingdom"]
uk_sales.head()

In [ ]:
date_total_uk = uk_sales.groupby("date")["date", "total"].agg("sum")
date_total_uk = date_total_uk.resample("M").sum()
date_total_uk.head()

In [ ]:
date_total_uk.plot(figsize=(25, 5))

from above graph between the **date vs total** of **uk_sales**
and from the graph between the **date vs total** in **sales** data.
It is clear that **United kingdom** data mostly affects on the total companies data.

clearly the sales max growth is incresing day by day.  

In [ ]:
date_customer_uk = (uk_sales.groupby("date")["date", "customer id"].agg("nunique")).drop("date", axis=1)
date_customer_uk.head()

In [ ]:
uk_final_sample = date_customer_uk.resample("M").mean()
uk_final_sample.rename(columns={"customer id":"customer count"}, inplace=True)
uk_final_sample["customer count"] = uk_final_sample["customer count"].astype(int)
uk_final_sample.head()

In [ ]:
uk_final_sample.plot(figsize=(25, 5))

In [ ]:
date_customer_uk.plot(figsize=(25, 5))

In [ ]:
uk_sales["date"] = pd.to_datetime(uk_sales["date"])

In [ ]:
# cats = ['Jan', 'Feb', 'Mar', 'Apr','May','Jun', 'Jul', 'Aug','Sep', 'Oct', 'Nov', 'Dec']
monthly_sales_uk = (uk_sales.groupby(uk_sales["date"].dt.strftime('%B'))["date", "customer id"].agg("nunique").drop("date", axis=1))

# monthly_sales_uk.index = pd.CategoricalIndex(monthly_sales_uk.index, categories=cats, ordered=True)
monthly_sales_uk

In [ ]:
monthly_sales_uk.plot.bar(figsize=(25, 5))

In [ ]:
uk_sales.columns

In [ ]:
uk_sales_resample = uk_sales[["date", "customer id", "quantity sold", "unit price", "total"]]

uk_sales_resample.head()


In [ ]:
sales.head()

In [ ]:
sales_group = sales.groupby("customer id").agg({"transaction country":"unique", "total":"sum"}).reset_index()
sales_group["transaction country"] = sales_group["transaction country"].apply(lambda x:x[0])
sales_group.head()

In [ ]:
country_customer = sales_group.groupby("transaction country").agg({"customer id":"nunique"})
country_customer.sort_values(by="customer id", ascending=False, inplace=True)
country_customer.head()

In [ ]:
sales.head()

In [ ]:
product_group = sales.groupby("product description").agg({"quantity sold":"sum"})
product_group.sort_values("quantity sold", ascending=False, inplace=True)
product_quantity_sold = product_group.head(20)
product_quantity_sold.head(10)

In [ ]:
product_quantity_sold.plot.bar(figsize=(25, 5))
plt.xticks(rotation = 45)

From above dataset i.e. **product_quantity_sold** it is clear that mostly decorative products customers are buying. So we can focus on decorative type of product to increase the sales.

Maybe mostly women's are doing shopping on our site. So we can also focus on women's product.
And the sales rate are increasing in last 4 month i.e. Sep, Oct, Nov and Dec.

In [ ]:
product_1 = sales[sales["product description"]=="PAPER CRAFT , LITTLE BIRDIE"]
product_1_count_date = product_1.groupby("date").agg({"quantity sold":"sum"})
product_1_count_month = product_1.groupby("month_of_year").agg({"quantity sold":"sum"}).sort_values("quantity sold", ascending=False)
product_1_count_month.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)

In [ ]:
product_2 = sales[sales["product description"]=="MEDIUM CERAMIC TOP STORAGE JAR"]
product_2_count_date = product_2.groupby("date").agg({"quantity sold":"sum"})
product_2_count_month = product_2.groupby("month_of_year").agg({"quantity sold":"sum"}).sort_values("quantity sold", ascending=False)
product_2_count_month.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)
product_2.head()

In [ ]:
product_3 = sales[sales["product description"]=="WORLD WAR 2 GLIDERS ASSTD DESIGNS"]
product_3_count_date = product_3.groupby("date").agg({"quantity sold":"sum"})
product_3_count_month = product_3.groupby("month_of_year").agg({"quantity sold":"sum"})
product_3_count_month.sort_values("quantity sold", ascending=False, inplace=True)
product_3_count_month.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)

In [ ]:
product_3 = sales[sales["product description"]=="JUMBO BAG RED RETROSPOT"]
product_3_count_date = product_3.groupby("date").agg({"quantity sold":"sum"})
product_3_count_month = product_3.groupby("month_of_year").agg({"quantity sold":"sum"}).sort_values("quantity sold", ascending=False)
product_3_count_month.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)

In [ ]:

product_3 = sales[sales["product description"]=="RABBIT NIGHT LIGHT"]
product_3_count_date = product_3.groupby("date").agg({"quantity sold":"sum"})
product_3_count_month = product_3.groupby("month_of_year").agg({"quantity sold":"sum"}).sort_values("quantity sold", ascending=False)
product_3_count_month.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)

In [ ]:
different_customer = sales.groupby("product description").agg({"customer id":"nunique"}).sort_values("customer id", ascending=False).head(20)
different_customer.plot.bar(figsize=(25, 5))
plt.xticks(rotation=45)

**Use of FBProphet**

In [ ]:
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot
from fbprophet.plot import plot_components

In [ ]:
datewise_total_reset = sales.groupby("date").agg({"total":"sum"})
datewise_total_reset = datewise_total_reset.reset_index()
datewise_total_reset.rename(columns={"date":"ds", "total":"y"}, inplace=True)
datewise_total_reset.tail()

In [ ]:
datewise_total_reset.dtypes

In [ ]:
test_datewise_total = pd.date_range(start="12/10/2011", end="31/1/2012")
test_datewise_total = pd.DataFrame(test_datewise_total, columns= ["ds"])
test_datewise_total.head()

In [ ]:
fb_model = Prophet()
fb_model.add_seasonality(name="monthly", period=7, fourier_order=5)
fb_model.add_country_holidays(country_name='UK')
fb_model.fit(datewise_total_reset)

In [ ]:
fb_model.train_holiday_names

In [ ]:
forecast = fb_model.predict(test_datewise_total)
forecast_result = forecast[["ds", "yhat"]]
forecast_result.rename(columns={"ds":"date", "yhat":"total"}, inplace=True)
forecast_result.set_index("date", inplace=True)
forecast_result.head()

In [ ]:
datewise_total = sales.groupby("date").agg({"total":"sum"})
datewise_total.tail()

In [ ]:
df = pd.concat([datewise_total, forecast_result], axis=0, sort=False)
df = df.resample("M").sum()
df.plot(figsize=(15, 5))

In [ ]:
forecast_train = fb_model.predict(datewise_total_reset)
forecast_train.head()

In [ ]:
forecast_result = pd.concat([forecast_train, forecast], axis=0, sort=False)

In [ ]:
fig = plot_components(fb_model, forecast_result, figsize=(10, 5))
ax = fig.gca()
ax.tick_params(axis="x", labelsize=15, rotation=45)
ax.tick_params(axis="y", labelsize=15)

In [ ]:
fig = plot(fb_model, forecast, figsize=(20, 7))
ax = fig.gca()
ax.set_title("date vs total", size=25)
ax.set_xlabel("date", size=15)
ax.set_ylabel("total", size=15)
ax.tick_params(axis="x", labelsize=15, rotation=45)
ax.tick_params(axis="y", labelsize=15)

In [ ]:
forecast.head()

In [ ]:
forecast_yhat = forecast[["ds", "yhat"]]
forecast_yhat.rename(columns={"yhat":"y"}, inplace=True)
forecast_yhat.head()

In [ ]:
df = pd.concat([datewise_total_reset, forecast_yhat], axis=0, sort=False)
df.rename(columns={"ds":"date", "y":"total"}, inplace=True)
df.set_index("date", inplace=True)
df.head()

In [ ]:
df_resample = df.resample("M").sum()
df_resample.plot(figsize=(15, 5))

**What kind of customer typically buy from us?**

In [ ]:
customer_count = sales.groupby("transaction country").agg({"customer id":"nunique"})
customer_count[customer_count.index=="United Kingdom"].values[0][0]

In [ ]:
customer_count = sales.groupby("transaction country").agg({"customer id":"nunique"})
print("total customers : ", customer_count["customer id"].sum())
print("total customers in UK : ",customer_count[customer_count.index=="United Kingdom"].values[0][0])

In [ ]:
customer_count = sales.groupby("transaction country").agg({"customer id":"nunique"})
customer_count = customer_count.sort_values("customer id", ascending=False).head(7)
customer_count.rename(columns={"customer id":"customer count"}, inplace=True)
customer_count.plot.bar(figsize=(15, 5))

**United Kingdom**

In [ ]:
sales_uk = sales[sales["transaction country"]=="United Kingdom"]
product_uk = sales_uk.groupby("product description").agg({"quantity sold":"sum"})
sort_uk_product = product_uk.sort_values("quantity sold", ascending=False).head(10)
sort_uk_product

In [ ]:
sales_uk = sales[sales["transaction country"]=="United Kingdom"]
product_uk = sales_uk.groupby("product description").agg({"quantity sold":"sum"})
sort_uk_product = product_uk.sort_values("quantity sold", ascending=False).head(10)
sort_uk_product.plot.bar(figsize=(15, 5))
plt.xticks(rotation=45)

**Germany**

In [ ]:
sales_ge = sales[sales["transaction country"]=="Germany"]
product_ge = sales_ge.groupby("product description").agg({"quantity sold":"sum"})
product_ge.sort_values("quantity sold", ascending=False).head(10)

In [ ]:
sales.head()

In [ ]:
customer_date = sales.groupby("customer id").agg({"date":"nunique"})
customer_date.rename(columns={"date":"num of times shopping"}, inplace=True)
customer_date.sort_values("num of times shopping", ascending=False).head(10)